In [1]:
# Install required packages.
import os
import torch
os.environ['TORCH'] = torch.__version__
os.environ['DGLBACKEND'] = "pytorch"

# Uncomment below to install required packages. If the CUDA version is not 11.8,
# check the https://www.dgl.ai/pages/start.html to find the supported CUDA
# version and corresponding command to install DGL.
#!pip install dgl -f https://data.dgl.ai/wheels/cu118/repo.html > /dev/null
#!pip install torchmetrics > /dev/null

try:
    import dgl
    installed = True
except ImportError:
    installed = False
print("DGL installed!" if installed else "Failed to install DGL!")

/Users/tangting/opt/anaconda3/envs/py36/lib/python3.6/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Failed to install DGL!


In [11]:
import os
from random import random
import sys

import numpy as np
import pandas as pd


import random 
from tqdm import tqdm

from GCN import  GCN
import argparse

import dgl
import dgl.nn as dglnn

import torch
import torch.nn as nn
import torch.nn.functional as F
from dgl import AddSelfLoop
from dgl.data import CiteseerGraphDataset, CoraGraphDataset, PubmedGraphDataset


DGL backend not selected or invalid.  Assuming PyTorch for now.


Setting the default backend to "pytorch". You can change it in the ~/.dgl/config.json file or export the DGLBACKEND environment variable.  Valid options are: pytorch, mxnet, tensorflow (all lowercase)


/Users/tangting/opt/anaconda3/envs/py39dgl/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# 仿真数据

In [6]:

# 节点
vertex_count = 1000
origin_ids = []
int_fetaures = []
float_fetaures = []
node_label_0 = []
node_label_1 = []
for i in range(vertex_count):
    origin_id = random.randint(1000000,100000000)
    if origin_id not in origin_ids:
        origin_ids.append(origin_id)
        int_fetaures.append([random.randint(10,100),random.randint(200,300)])
        float_fetaures.append([random.random()*10,random.random()*2000])
        node_label_0.append(int(random.random()*2))    
        node_label_1.append(int(random.random()*2))



# vertext_tb = pd.DataFrame(data=[origin_ids,int_fetaures,float_fetaures],columns=["origin_id","int_fetaures","float_fetaures"])
vertext_tb = pd.DataFrame({"origin_id" : origin_ids, \
                            "int_fetaures": int_fetaures, \
                            "float_fetaures" :float_fetaures,
                            "node_label_0":node_label_0,\
                           "node_label_1":node_label_1     })
vertext_tb["id"] = vertext_tb.index
vertext_tb.to_csv("data/vertex_data.csv",index=False,header=True)
vertext_tb

,origin_id,int_fetaures,float_fetaures,node_label_0,node_label_1,id
0,5125489,"[94, 210]","[7.75003028894565, 687.4913727952168]",1,1,0
1,79364113,"[72, 265]","[4.947234765668654, 1192.0257425240652]",1,1,1
2,73107019,"[80, 213]","[8.931735387205725, 1581.4020891830394]",0,0,2
3,76844737,"[69, 254]","[4.91752437767527, 952.87631501277]",0,0,3
4,5457649,"[23, 294]","[7.523821229407282, 1345.7618291077938]",0,1,4
...,...,...,...,...,...,...
995,87095250,"[30, 202]","[8.604962517372929, 830.3876275191404]",0,1,995
996,15823269,"[15, 292]","[9.051344293880758, 867.5854277403004]",1,0,996
997,37451013,"[70, 250]","[0.4214429922410845, 163.89009599979]",0,0,997
998,44034545,"[72, 242]","[8.222930254721824, 1463.1290476501986]",1,0,998


In [31]:
vertex_count = len(origin_ids)
edge_count = vertex_count * 10
edge_src_ids = []
edge_dst_ids = []
edge_types = []
for i in range(edge_count):
    src_rand_id = random.randint(0,vertex_count-1)
    dst_rand_id = random.randint(0,vertex_count-1)
    if src_rand_id != dst_rand_id:
        src_id = origin_ids[src_rand_id]
        edge_src_ids.append(src_id)
        dst_id = origin_ids[dst_rand_id]
        edge_dst_ids.append(dst_id)
        edge_types.append(random.randint(0,5))

# vertext_tb = pd.DataFrame(data=[origin_ids,int_fetaures,float_fetaures],columns=["origin_id","int_fetaures","float_fetaures"])
edge_tb = pd.DataFrame({"src_origin_id" : edge_src_ids,"dst_origin_id": edge_dst_ids,"edge_type" :edge_types})

edge_tb.to_csv("data/edge_data.csv",index=False,header=True)

In [32]:
edge_tb = pd.merge(edge_tb,vertext_tb[["origin_id","id"]],left_on="src_origin_id",right_on="origin_id",how="left") .rename(columns={"id":"src_id"}).drop(columns=["origin_id"])
edge_tb = pd.merge(edge_tb,vertext_tb[["origin_id","id"]],left_on="dst_origin_id",right_on="origin_id",how="left") .rename(columns={"id":"dst_id"}).drop(columns=["origin_id"])
edge_tb

,src_origin_id,dst_origin_id,edge_type,src_id,dst_id
0,88152249,80400612,5,351,178
1,88214274,92154276,3,963,461
2,51564490,30499255,4,291,311
3,14762157,45052990,0,616,797
4,54140698,86187938,0,396,302
...,...,...,...,...,...
9981,49029040,40926498,4,806,940
9982,77531573,66938313,2,594,981
9983,19734122,84579240,3,532,742
9984,18270251,66938313,2,79,981


In [60]:
def _preprocess_features(feature):
    return feature



In [33]:
# 加载成图

u,v = edge_tb["src_id"].tolist(),edge_tb["dst_id"].tolist()
g = dgl.graph((u, v))

In [69]:
g.ndata['int_feat'] = torch.tensor(_preprocess_features(vertext_tb["int_fetaures"]))
g.ndata['float_feat'] = torch.tensor(_preprocess_features(vertext_tb["float_fetaures"]))

g.ndata['pre_label'] = torch.tensor(vertext_tb["node_label_0"],dtype=torch.float)
g.ndata['label'] = torch.tensor(vertext_tb["node_label_1"],dtype=torch.float)

g.edata['edge_type'] = torch.tensor(edge_tb["edge_type"],dtype=torch.float)

In [70]:
g.ndata["train_mask"] = torch.randint(0,2,[g.ndata['label'].shape[0],1])

g.ndata["val_mask"] = torch.randint(0,2,[g.ndata['label'].shape[0],1])
g.ndata["test_mask"] = torch.randint(0,2,[g.ndata['label'].shape[0],1])

In [71]:
g.ndata["feat"] = torch.concat([g.ndata['int_feat'],g.ndata['float_feat'],torch.reshape(g.ndata['pre_label'],[-1,1])],axis =1)

In [72]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
g = g.int().to(device)
features = g.ndata["feat"]
labels = g.ndata["label"]
masks = g.ndata["train_mask"], g.ndata["val_mask"], g.ndata["test_mask"]

# create GCN model
in_size = features.shape[1]
out_size = 1
model = GCN(in_size, 16, out_size).to(device)

In [73]:



def evaluate(g, features, labels, mask, model):
    model.eval()
    with torch.no_grad():
        logits = model(g, features)
        logits = logits[mask]
        labels = labels[mask]
        _, indices = torch.max(logits, dim=1)
        correct = torch.sum(indices == labels)
        return correct.item() * 1.0 / len(labels)


def train(g, features, labels, masks, model):
    # define train/val samples, loss function and optimizer
    train_mask = masks[0]
    val_mask = masks[1]
    loss_fcn = nn.MSELoss()
    sigmoid = nn.Sigmoid()
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-2, weight_decay=5e-4)

    # training loop
    for epoch in range(200):
        model.train()
        logits = model(g, features)
        pred = sigmoid(logits)
        loss = loss_fcn(pred[train_mask], labels[train_mask])
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        acc = evaluate(g, features, labels, val_mask, model)
        print(
            "Epoch {:05d} | Loss {:.4f} | Accuracy {:.4f} ".format(
                epoch, loss.item(), acc
            )
        )


# model training
print("Training...")
train(g, features, labels, masks, model)

# test the model
print("Testing...")
acc = evaluate(g, features, labels, masks[2], model)
print("Test accuracy {:.4f}".format(acc))

        

Training...
Epoch 00000 | Loss 0.5010 | Accuracy 0.0000 
Epoch 00001 | Loss 1.0000 | Accuracy 0.0000 
Epoch 00002 | Loss 1.0000 | Accuracy 0.0000 
Epoch 00003 | Loss 1.0000 | Accuracy 0.0000 
Epoch 00004 | Loss 0.4990 | Accuracy 0.0000 
Epoch 00005 | Loss 1.0000 | Accuracy 0.0000 
Epoch 00006 | Loss 1.0000 | Accuracy 0.0000 
Epoch 00007 | Loss 1.0000 | Accuracy 0.0000 
Epoch 00008 | Loss 1.0000 | Accuracy 0.0000 
Epoch 00009 | Loss 1.0000 | Accuracy 0.0000 
Epoch 00010 | Loss 0.9998 | Accuracy 0.0000 
Epoch 00011 | Loss 1.0000 | Accuracy 0.0000 
Epoch 00012 | Loss 0.9999 | Accuracy 0.0000 
Epoch 00013 | Loss 0.5010 | Accuracy 0.0000 
Epoch 00014 | Loss 0.5037 | Accuracy 0.0000 
Epoch 00015 | Loss 0.4990 | Accuracy 0.0000 
Epoch 00016 | Loss 1.0000 | Accuracy 0.0000 


/Users/tangting/opt/anaconda3/envs/py39dgl/lib/python3.9/site-packages/torch/nn/modules/loss.py:530: UserWarning: Using a target size (torch.Size([1000, 1])) that is different to the input size (torch.Size([1000, 1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/Users/tangting/opt/anaconda3/envs/py39dgl/lib/python3.9/site-packages/torch/amp/autocast_mode.py:198: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


Epoch 00017 | Loss 0.5010 | Accuracy 0.0000 
Epoch 00018 | Loss 1.0000 | Accuracy 0.0000 
Epoch 00019 | Loss 0.0000 | Accuracy 0.0000 
Epoch 00020 | Loss 0.5010 | Accuracy 0.0000 
Epoch 00021 | Loss 0.5010 | Accuracy 0.0000 
Epoch 00022 | Loss 0.0000 | Accuracy 0.0000 
Epoch 00023 | Loss 0.0000 | Accuracy 0.0000 
Epoch 00024 | Loss 0.0000 | Accuracy 0.0000 
Epoch 00025 | Loss 0.5010 | Accuracy 0.0000 
Epoch 00026 | Loss 0.0000 | Accuracy 0.0000 
Epoch 00027 | Loss 0.0000 | Accuracy 0.0000 
Epoch 00028 | Loss 0.0000 | Accuracy 0.0000 
Epoch 00029 | Loss 0.0000 | Accuracy 0.0000 
Epoch 00030 | Loss 0.0000 | Accuracy 0.0000 
Epoch 00031 | Loss 0.0000 | Accuracy 0.0000 
Epoch 00032 | Loss 0.0000 | Accuracy 0.0000 
Epoch 00033 | Loss 0.0000 | Accuracy 0.0000 
Epoch 00034 | Loss 0.0000 | Accuracy 0.0000 
Epoch 00035 | Loss 0.0000 | Accuracy 0.0000 
Epoch 00036 | Loss 0.0000 | Accuracy 0.0000 
Epoch 00037 | Loss 0.0000 | Accuracy 0.0000 
Epoch 00038 | Loss 0.0000 | Accuracy 0.0000 
Epoch 0003

Package              Version
-------------------- ---------
absl-py              1.1.0
appnope              0.1.3
astor                0.8.1
backcall             0.2.0
certifi              2021.5.30
cloudpickle          2.1.0
cycler               0.11.0
dataclasses          0.8
decorator            4.4.2
dowhy                0.7.1
econml               0.13.1
entrypoints          0.4
gast                 0.2.2
google-pasta         0.2.0
grpcio               1.46.3
h5py                 2.10.0
importlib-metadata   4.8.3
importlib-resources  5.4.0
ipykernel            5.5.6
ipython              7.16.3
ipython-genutils     0.2.0
jedi                 0.17.2
joblib               1.1.0
jupyter-client       7.1.2
jupyter-core         4.9.2
Keras-Applications   1.0.8
Keras-Preprocessing  1.1.2
kiwisolver           1.3.1
lightgbm             3.3.2
llvmlite             0.36.0
Markdown             3.3.7
matplotlib           3.3.4
mpmath               1.2.1
nest-asyncio         1.5.5
networkx       